<a href="https://colab.research.google.com/github/debabratabepari/Data-Science-Practice/blob/main/diff_vectorization_spamham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("SMSSpamCollection.csv")

In [3]:
df.head()

,label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5574, 2)

In [5]:
import re
import nltk
from nltk.corpus import stopwords
#downloading stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# stopwords.words('english')

In [7]:
df["SMS"][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [8]:
#Missing value checking
df.isnull().sum()

label    0
SMS      0
dtype: int64

In [9]:
#Stemming package
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [10]:
# Creating Corpus by removing unnecessary elements, lowering, stemming and using stopwords
corpus=[]
for i in range(0, len(df)):
  review = re.sub("[^a-zA-Z0-9]"," ", df["SMS"][i]) #removing unnecessary elements by only having a-z and 0-9
  review = review.lower()  #lowering each sentences
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #stemming and stopwords
  review = ' '.join(review)
  corpus.append(review)


In [11]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6day 16 tsandc appli repli hl 4 info',
 'urgent 1 week free mem

### Bag of words

In [12]:
# creating Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 2500, binary= True, ngram_range= (1,3))
x= cv.fit_transform(corpus).toarray()

In [13]:
x.shape

(5574, 2500)

In [14]:
# Label encoding
y= pd.get_dummies(df['label'])
# y

In [15]:
y = y.iloc[:,1].values.astype(int)
y

array([0, 0, 1, ..., 0, 0, 0])

In [16]:
# train_test_split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Naive Bayes Model

In [17]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train,y_train)

In [18]:
y_pred = model.predict(x_test)

In [19]:
from sklearn.metrics import accuracy_score, classification_report
score = accuracy_score(y_test,y_pred)
score

0.9790794979079498

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1447
           1       0.93      0.92      0.92       226

    accuracy                           0.98      1673
   macro avg       0.96      0.95      0.95      1673
weighted avg       0.98      0.98      0.98      1673



### TF-IDF

In [21]:
# Creating TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 2500, ngram_range=(1,3))
x = tv.fit_transform(corpus).toarray()

In [22]:
x.shape

(5574, 2500)

In [23]:
x,y

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 1, ..., 0, 0, 0]))

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state= 50)

Naive Bayes model

In [25]:
from sklearn.naive_bayes import MultinomialNB
model_tv = MultinomialNB().fit(x_train, y_train)

In [26]:
y_pred = model_tv.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1])

In [27]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

0.97847533632287
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       957
           1       1.00      0.85      0.92       158

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



Random Forest model

In [28]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(x_train,y_train)

RandomForestClassifier()

In [29]:
y_pred = model_rf.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1])

In [30]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

0.9829596412556054
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       957
           1       1.00      0.88      0.94       158

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



# Word2vec

In [31]:
# !pip install gensim

In [32]:
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')
# wv

In [33]:
# Applying leammatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [34]:
corpus=[]
for i in range(0, len(df)):
  review = re.sub("[^a-zA-Z0-9]"," ", df["SMS"][i])
  review = review.lower()
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6days 16 tsandcs apply reply

In [35]:
len(corpus)

5574

In [36]:
from nltk import sent_tokenize
from nltk import word_tokenize
from gensim.utils import simple_preprocess

In [37]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [38]:
# !pip install nltk

In [39]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
# Tokenization
words=[]
for sent in corpus:
  sent_token = word_tokenize(sent)
  # for sent in sent_token:
    # words.append(simple_preprocess(sent))
  words.append(sent_token)

In [41]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'u', 'oni'],
 ['free',
  'entry',
  '2',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  '21st',
  'may',
  '2005',
  'text',
  'fa',
  '87121',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'c',
  'apply',
  '08452810075over18'],
 ['u', 'dun', 'say', 'early', 'hor', 'u', 'c', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  '3',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  '1',
  '50',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  '

In [42]:
# implementing Word2vec model
import gensim
model = gensim.models.Word2Vec(words, window= 5, min_count= 2)

In [ ]:
# Vocabulary
model.wv.index_to_key

In [43]:
s=len(model.wv.index_to_key)

In [44]:
model.corpus_count

5574

In [45]:
model.epochs

5

In [46]:
# similar word identifying
model.wv.similar_by_word('water')

[('meet', 0.995719313621521),
 ('phone', 0.9957073926925659),
 ('person', 0.9956573843955994),
 ('put', 0.9956186413764954),
 ('thought', 0.9956070184707642),
 ('make', 0.9955593347549438),
 ('smile', 0.9955301880836487),
 ('u', 0.9955300688743591),
 ('much', 0.9955182075500488),
 ('great', 0.9955154657363892)]

In [47]:
model.wv['shop'].shape

(100,)

In [48]:
# Creating Average Word2vec function
import numpy as np

def avg_word2vec(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [49]:
# !pip install tqdm

In [50]:
from tqdm import tqdm

In [51]:
words[960]

[]

In [52]:
# Implementing Average Word2Vec Model
l = []
for i in tqdm(range(len(words))):
  l.append(avg_word2vec(words[i]))

  0%|          | 0/5574 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5574/5574 [00:01<00:00, 4649.07it/s]


In [53]:
# Counting "nan" value present
import math
sum(math.isnan(x) for x in l if isinstance(x, float))

23

In [54]:
# Getting the indices corresponding to "nan" value
nan_count=0
nan_indices=[]
for i, value in enumerate(l):
    if isinstance(value, float) and math.isnan(value):
        nan_count += 1
        nan_indices.append(i)



In [55]:
nan_indices,nan_count

([253,
  451,
  509,
  783,
  893,
  960,
  1445,
  1592,
  1612,
  1836,
  1995,
  2415,
  2570,
  2807,
  3376,
  3740,
  3852,
  4117,
  4286,
  4480,
  4575,
  4824,
  4887],
 23)

In [56]:
l[960]

nan

In [57]:
# Removing "nan" value
cleaned_list = [x for x in l if not isinstance(x, float) or not math.isnan(x)]

In [58]:
cleaned_list[0]

array([-0.18452704,  0.31771502,  0.16760156,  0.03282069,  0.04236975,
       -0.427859  ,  0.12900107,  0.68859375, -0.1671468 , -0.09793471,
       -0.13882715, -0.5359903 , -0.15811907,  0.07892898,  0.04904805,
       -0.15912457, -0.01404617, -0.42142582,  0.01774891, -0.54026717,
        0.14935799,  0.16335218,  0.0999329 , -0.20099665, -0.0140135 ,
        0.08141278, -0.20421875, -0.2303462 , -0.21920781,  0.00737266,
        0.29264182,  0.16995394,  0.0745365 , -0.2005299 , -0.16534884,
        0.50534284,  0.09684087, -0.33265582, -0.2239354 , -0.6052026 ,
       -0.03828583, -0.36272794, -0.09453236,  0.02029926,  0.19619031,
       -0.09876013, -0.25162625, -0.06568012,  0.13793238,  0.19276316,
        0.14409383, -0.28128275, -0.15536547,  0.0456506 , -0.18372391,
        0.16082828,  0.11966579, -0.01955459, -0.3816707 ,  0.20739672,
        0.19687419,  0.11989051, -0.08985962, -0.09947594, -0.34709802,
        0.15130106,  0.09030392,  0.2388298 , -0.3797283 ,  0.43

In [59]:
l_new = np.array(cleaned_list)

In [60]:
l_new.shape

(5551, 100)

In [61]:
l_new

array([[-0.18452704,  0.31771502,  0.16760156, ..., -0.28830108,
         0.03635314, -0.13625154],
       [-0.16373487,  0.2778797 ,  0.14873417, ..., -0.25806051,
         0.03644554, -0.11947771],
       [-0.150115  ,  0.26135215,  0.13931668, ..., -0.24089287,
         0.02951808, -0.11164651],
       ...,
       [-0.02726475,  0.03708509,  0.0235791 , ..., -0.03377822,
         0.00745001, -0.01800238],
       [-0.19623499,  0.33375964,  0.17898242, ..., -0.30985704,
         0.04056698, -0.14366409],
       [-0.11435121,  0.1853335 ,  0.10462538, ..., -0.17743444,
         0.02206128, -0.08007454]], dtype=float32)

In [62]:
# Deleting the target values corresponding to "nan" indices
y_new = np.delete(y,nan_indices)

In [63]:
y_new.shape

(5551,)

In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(l_new, y_new, test_size = 0.2, random_state = 42)

Random Forest Model

In [65]:
from sklearn.ensemble import RandomForestClassifier
model_wv_rf = RandomForestClassifier()
model_wv_rf.fit(x_train,y_train)

RandomForestClassifier()

In [66]:
y_pred = model_wv_rf.predict(x_test)
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [67]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9477947794779478
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       937
           1       0.96      0.70      0.81       174

    accuracy                           0.95      1111
   macro avg       0.95      0.85      0.89      1111
weighted avg       0.95      0.95      0.94      1111



LSTM RNN Model

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding

In [69]:
x_lstm = l_new.reshape(5551,1,100)

In [70]:
x_train, x_test, y_train, y_test = train_test_split(x_lstm, y_new, test_size = 0.3, random_state = 50)

In [71]:
x_train.shape

(3885, 1, 100)

In [72]:
# Creating LSTM RNN model
model = Sequential()
model.add(LSTM(150, input_shape=(1,100),dropout= 0.2, recurrent_dropout= 0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = "sigmoid"))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [73]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 64)                9664      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 160329 (626.29 KB)
Trainable params: 160329 (626.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Model Training

In [74]:
 model.fit(x_train,y_train,validation_data= (x_test,y_test), epochs=20, batch_size= 64)

Epoch 1/20
61/61 [==============================] - 4s 16ms/step - loss: 0.4721 - accuracy: 0.8538 - val_loss: 0.4177 - val_accuracy: 0.8673
Epoch 2/20
61/61 [==============================] - 1s 9ms/step - loss: 0.4250 - accuracy: 0.8646 - val_loss: 0.4090 - val_accuracy: 0.8673
Epoch 3/20
61/61 [==============================] - 1s 9ms/step - loss: 0.4213 - accuracy: 0.8646 - val_loss: 0.4025 - val_accuracy: 0.8673
Epoch 4/20
61/61 [==============================] - 1s 10ms/step - loss: 0.4114 - accuracy: 0.8646 - val_loss: 0.3985 - val_accuracy: 0.8673
Epoch 5/20
61/61 [==============================] - 1s 10ms/step - loss: 0.4084 - accuracy: 0.8646 - val_loss: 0.3971 - val_accuracy: 0.8673
Epoch 6/20
61/61 [==============================] - 1s 10ms/step - loss: 0.4063 - accuracy: 0.8646 - val_loss: 0.3919 - val_accuracy: 0.8673
Epoch 7/20
61/61 [==============================] - 1s 11ms/step - loss: 0.4008 - accuracy: 0.8646 - val_loss: 0.3891 - val_accuracy: 0.8673
Epoch 8/20
61/6

In [75]:
model.evaluate(x_test,y_test)

53/53 [==============================] - 0s 3ms/step - loss: 0.3293 - accuracy: 0.8673


[0.32929834723472595, 0.8673469424247742]

In [81]:
y_pred = model.predict(x_test)
y_pred

53/53 [==============================] - 0s 3ms/step


array([[0.2040668 ],
       [0.05801034],
       [0.26171628],
       ...,
       [0.1305622 ],
       [0.10996828],
       [0.13078782]], dtype=float32)

In [77]:
# ROC_AUC curve
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Create ROC curve plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Random Guess'))

# Update layout
fig.update_layout(
    title='Receiver Operating Characteristic (ROC)',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    legend=dict(x=0.05, y=0.95),
    margin=dict(l=20, r=20, t=60, b=20),
    hovermode='closest'
)

# Show plot
fig.show()

In [82]:
y_pred = np.where(y_pred>0.87,1,0)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [83]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[1445    0]
 [ 221    0]]


In [84]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8673469387755102
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1445
           1       0.00      0.00      0.00       221

    accuracy                           0.87      1666
   macro avg       0.43      0.50      0.46      1666
weighted avg       0.75      0.87      0.81      1666



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

